In [104]:
import pandas as pd #数据分析
import numpy as np #科学计算
from pandas import Series,DataFrame
from sklearn.model_selection import train_test_split

df = pd.read_csv("/Users/wangweiqi/Desktop/Titanic.csv") #运用pandas读取csv文件


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [106]:
df.describe() #查看数据的描述性统计，文本信息和类目信息不显示

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [107]:
from sklearn.ensemble import RandomForestRegressor

In [108]:
def set_missing_ages(df):

    # 把已有的数值型特征取出来丢进Random Forest Regressor中
    age_df = df[['Age','Fare', 'Parch', 'SibSp', 'Pclass']]

    # 乘客分成已知年龄和未知年龄两部分
    known_age = age_df[age_df.Age.notnull()].as_matrix()
    unknown_age = age_df[age_df.Age.isnull()].as_matrix()

    # y即目标年龄
    y = known_age[:, 0]

    # X即特征属性值
    X = known_age[:, 1:]

    # fit到RandomForestRegressor之中
    rfr = RandomForestRegressor(random_state=0, n_estimators=2000, n_jobs=-1)
    rfr.fit(X, y)

    # 用得到的模型进行未知年龄结果预测
    predictedAges = rfr.predict(unknown_age[:, 1:])
    

    # 用得到的预测结果填补原缺失数据
    df.loc[df.Age.isnull(), 'Age'] = predictedAges 

    return df, rfr

In [109]:
def proc(str1):
    res = str1.split(',')[1].split('.')[0].strip()
    if res in ['Mr', 'Miss', 'Mrs', 'Master']:
        return res
    else:
        return 'Others'

df['Title']=df['Name'].astype(str).apply(proc)

In [110]:
df['HasCabin']=df['Cabin'].apply(pd.isnull) 

In [111]:
df, rfr = set_missing_ages(df)

In [112]:
dummies_HasCabin = pd.get_dummies(df['HasCabin'], prefix= 'HasCabin')
dummies_Embarked = pd.get_dummies(df['Embarked'], prefix= 'Embarked')
dummies_Sex = pd.get_dummies(df['Sex'], prefix= 'Sex')
dummies_Pclass = pd.get_dummies(df['Pclass'], prefix= 'Pclass')
dummies_Title = pd.get_dummies(df['Title'], prefix= 'Title')

df = pd.concat([df, dummies_HasCabin, dummies_Embarked, dummies_Sex, dummies_Pclass,dummies_Title], axis=1)
df.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'HasCabin', 'Embarked','Title',], axis=1, inplace=True)


In [113]:
import sklearn.preprocessing as preprocessing

In [114]:
scaler = preprocessing.StandardScaler()
age_scale_param = scaler.fit(df['Age'].values.reshape(-1, 1))

In [115]:
df['Age_scaled'] = scaler.fit_transform(df['Age'].values.reshape(-1, 1), age_scale_param)
fare_scale_param = scaler.fit(df['Fare'].values.reshape(-1, 1))
df['Fare_scaled'] = scaler.fit_transform(df['Fare'].values.reshape(-1, 1), fare_scale_param)

In [116]:
from sklearn.model_selection import train_test_split

In [117]:
df_train, df_test = train_test_split(df, stratify=df['Survived'], test_size=0.2, random_state=42)

In [118]:
df_train.shape,df_test.shape

((712, 24), (179, 24))

In [119]:
df.columns

Index(['PassengerId', 'Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'HasCabin_False', 'HasCabin_True', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2',
       'Pclass_3', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
       'Title_Others', 'Age_scaled', 'Fare_scaled'],
      dtype='object')

In [120]:
from sklearn import linear_model

# 用正则取出我们要的属性值
df_train = df_train.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|HasCabin_.*|Embarked_.*|Sex_.*|Pclass_.*|Title_.*')
np_train = df_train.as_matrix()

# y即Survival结果
Y = np_train[:, 0]

# X即特征属性值
X = np_train[:, 1:]


# fit到LogisticRegression之中
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
clf.fit(X, Y)

clf.predict

<bound method LinearClassifierMixin.predict of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)>

In [121]:
df_train.shape

(712, 20)

In [122]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

prediction = clf.predict(X)
print ('Accuracy:', accuracy_score(Y, prediction))
print ('F1 score:', f1_score(Y, prediction))
print ('Recall:', recall_score(Y, prediction))
print ('Precision:', precision_score(Y, prediction))
print ('\n clasification report:\n', classification_report(Y,prediction))
print ('\n confussion matrix:\n',confusion_matrix(Y, prediction))

Accuracy: 0.839887640449
F1 score: 0.784905660377
Recall: 0.761904761905
Precision: 0.809338521401

 clasification report:
              precision    recall  f1-score   support

        0.0       0.86      0.89      0.87       439
        1.0       0.81      0.76      0.78       273

avg / total       0.84      0.84      0.84       712


 confussion matrix:
 [[390  49]
 [ 65 208]]


In [ ]:
#Accuracy：判断正确率
#precision：预测对的/预测的总体
#recall：预测对的/所有样本中对的
#F1 score：综合precision和recall


In [123]:
df_test = df_test.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|HasCabin_.*|Embarked_.*|Sex_.*|Pclass_.*|Title_.*')
np_test = df_test.as_matrix()
Y=np_test[:,0]
X=np_test[:,1:]

In [124]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

prediction = clf.predict(X)
print ('Accuracy:', accuracy_score(Y, prediction))
print ('F1 score:', f1_score(Y, prediction))
print ('Recall:', recall_score(Y, prediction))
print ('Precision:', precision_score(Y, prediction))
print ('\n clasification report:\n', classification_report(Y,prediction))
print ('\n confussion matrix:\n',confusion_matrix(Y, prediction))

Accuracy: 0.843575418994
F1 score: 0.794117647059
Recall: 0.782608695652
Precision: 0.805970149254

 clasification report:
              precision    recall  f1-score   support

        0.0       0.87      0.88      0.87       110
        1.0       0.81      0.78      0.79        69

avg / total       0.84      0.84      0.84       179


 confussion matrix:
 [[97 13]
 [15 54]]


In [125]:
pd.DataFrame({"columns":list(df_train.columns)[1:], "coef":list(clf.coef_.T)})

,coef,columns
0,[-0.419499891359],SibSp
1,[-0.286907889354],Parch
2,[1.03296611701],HasCabin_False
3,[-0.00982700367465],HasCabin_True
4,[0.0],Embarked_C
5,[0.163271827908],Embarked_Q
6,[-0.38756008732],Embarked_S
7,[1.64317713997],Sex_female
8,[-0.212063074266],Sex_male
9,[0.392898360186],Pclass_1
